In [1]:
# %% Test: Ingesta con datos inválidos
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from delta import *
import re

In [2]:
# Configuración Spark en modo local
master_url = "local[*]"

builder = (
    SparkSession.builder
    .appName("Lab_SECOP_TEST")
    .master(master_url)
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.executor.memory", "1g")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e40ffb42-9969-4444-97ce-2ecbeb28f674;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 218ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

In [3]:
# LECTURA CSV DE PRUEBA
print("Leyendo CSV de prueba con datos inválidos...")
csv_path = "/app/data/test_invalid_data.csv"

df_raw = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load(csv_path)
)

print(f"Registros leídos: {df_raw.count()}")
print(df_raw.show(15))

Leyendo CSV de prueba con datos inválidos...


26/02/02 05:26:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Registros leídos: 10
+-------------+-----------+------------+------+------+-----------------------+----------------+-------------------------+---------------------------------------+--------------+----------------------------+-------------------------+-----------+-----------+------------+
|      Entidad|Nit Entidad|Departamento|Ciudad|Estado|Descripcion del Proceso|Tipo de Contrato|Modalidad de Contratacion|Justificacion Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Precio Base|Valor Total|Valor Pagado|
+-------------+-----------+------------+------+------+-----------------------+----------------+-------------------------+---------------------------------------+--------------+----------------------------+-------------------------+-----------+-----------+------------+
|   ALCALDIA 1|       NULL|   ANTIOQUIA|  NULL|  NULL|                   NULL|            NULL|                     NULL|                                   NULL|    2023-01

In [4]:
# LIMPIEZA DE DATOS
def clean_name(c: str) -> str:
    c = c.strip()
    c = re.sub(r"[;:{}()\n\t\r]", "", c)
    c = re.sub(r"\s+", "_", c)
    c = re.sub(r"[^A-Za-z0-9_]", "", c)
    return c

cols_limpias = [clean_name(c) for c in df_raw.columns]

for old, new in zip(df_raw.columns, cols_limpias):
    if old != new:
        df_raw = df_raw.withColumnRenamed(old, new)

In [5]:
# ESCRITURA BRONZE DE PRUEBA
print("Escribiendo en capa Bronze de prueba...")
output_path = "data/lakehouse/bronze/test_secop"

(
    df_raw.write
    .format("delta")
    .mode("overwrite")
    .save(output_path)
)

print(f"✅ Ingesta completada. Registros: {df_raw.count()}")

Escribiendo en capa Bronze de prueba...


26/02/02 05:26:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 10:===============================================>        (42 + 5) / 50]

✅ Ingesta completada. Registros: 10
